In [1]:
import pandas as pd
import os
import sys

In [2]:
pd.__version__

'0.25.1'

In [33]:
#os.chdir('D:\Javier\SynologyDrive\Portfolio Python\Test')

In [37]:
folder = 'D:\Javier\SynologyDrive\Portfolio Python\Test\IBActivityNew'



In [5]:
tables = 'D:\Javier\SynologyDrive\Portfolio Python\Test\Tables'

## Functions

In [19]:
#Get all Forex
def get_all_forex(folder):
    #Creates a list of all files in folder.
    filelist = os.listdir(folder)
    #Creates a empty list to append all dataframes
    dataframe_list = []
    #Iterates through all files in filelist
    for file in filelist:
        #Checks if file is a .csv
        if file.lower().endswith('.csv'):
            # Creates a relative path from working directory to the current element of filelist
            file_path = os.path.join('IbActivityNew', file)
            # reads file and creates a dataframe forcing 17 columns
            data = pd.read_csv(file_path, names=list('abcdefghijklmnopq'))
            # Filters first column 'Trades' values
            data_trades = data[data.a == 'Trades']
            #Checks if any trades on file
            if len(data_trades) > 0:
                # Dataframe from the first row
                data_headers = data_trades.iloc[0]
                #Uncoment this line and related if necessary to make collumns single word. Comment out equivalent lines
                #data_headers = data_headers.str.replace(' ', '_')
                # Convert dataframe to list
                data_headers = list(data_headers)
                # Assigns values to Coumns           
                data_trades.columns = data_headers
                # Removes first row that now is duplicated
                trades = data_trades.drop(data_trades.index[0])
                #Filters data
                trades = trades[trades.Header == 'Data']
                trades = trades[trades['Asset Category'] == 'Forex']
                #trades = trades[data_trades['Asset_Category'] != 'Forex']
                #Check if any trades after filtering
                if len(trades) > 0:
                    # Remove columns with NaN values
                    trades = trades.dropna(axis=1)
                    #trades = trades.drop(columns=['Trades','Header','DataDiscriminator')
                    # Removes columns
                    delete_columns = ['Trades','Header','DataDiscriminator','Basis','Realized P/L','C. Price','MTM P/L','MTM in USD','Code']
                    for  i in  delete_columns:
                        if i in trades.columns:
                            trades = trades.drop(columns=i)             
                
                    #trades = trades.drop(columns=['Trades','Header','DataDiscriminator'])
                    # Fix some incorrect values on Quantity eg 2,000 -> 2000
                    trades['Quantity'] = trades['Quantity'].str.replace(',', '')
                    trades.rename(columns={'Comm in USD': 'Comm/Fee'}, inplace=True)
                    # Appends to dataframe list
                    dataframe_list.append(trades)
                # Checks if dataframe list is empty    
                if len(dataframe_list) > 0:
                # Concatenates all dataframes
                # Creates a dataframe with all new trades
                    dataframe = pd.concat(dataframe_list,sort=False ).reset_index(drop=True)
                    dataframe['Proceeds'] = dataframe['Proceeds'].astype('float')
                    dataframe['Quantity'] = dataframe['Quantity'].astype('float')
                    dataframe['T. Price'] = dataframe['T. Price'].astype('float')
                    dataframe['Comm/Fee'] = dataframe['Comm/Fee'].astype('float')
                    dataframe['Date/Time'] = pd.to_datetime(dataframe['Date/Time'])
                    dataframe['Quantity_Rsum'] =  dataframe.groupby(['Symbol'])['Quantity'].transform(lambda x : x.cumsum())

    return dataframe

In [38]:
all_forex = get_all_forex(folder)

In [21]:
#Save .csv
#all_forex.to_csv((os.path.join(tables,r'tables_forex.csv')))

In [39]:
all_forex

,Asset Category,Currency,Symbol,Date/Time,Quantity,T. Price,Proceeds,Comm/Fee,Quantity_Rsum
0,Forex,HKD,USD.HKD,2017-03-20 10:51:15,9000.0000,7.76574,-69891.660000,-2.00000,9000.0000
1,Forex,HKD,USD.HKD,2017-05-25 10:00:03,2000.0000,7.79131,-15582.620000,-2.00000,11000.0000
2,Forex,HKD,USD.HKD,2017-07-04 09:43:20,3845.0000,7.80751,-30019.875950,-2.00000,14845.0000
3,Forex,HKD,USD.HKD,2017-07-06 18:08:50,0.7482,7.81055,-5.843854,0.00000,14845.7482
4,Forex,HKD,USD.HKD,2017-11-21 09:55:29,3840.0000,7.81137,-29995.660800,-2.00000,18685.7482
5,Forex,HKD,USD.HKD,2017-11-23 18:47:55,0.5555,7.81135,-4.339205,0.00000,18686.3037
6,Forex,USD,EUR.USD,2017-11-28 22:09:23,20000.0000,1.18520,-23704.000000,-2.00000,20000.0000
7,Forex,USD,EUR.USD,2017-11-29 01:30:35,-20000.0000,1.18595,23719.000000,-2.00000,0.0000
8,Forex,USD,EUR.USD,2017-12-20 03:32:12,20000.0000,1.18482,-23696.400000,-2.00000,20000.0000
9,Forex,USD,EUR.USD,2017-12-20 10:19:13,-20000.0000,1.18780,23756.000000,-2.00000,0.0000


In [25]:
#Get all dividends from all activity statements in folder
def get_all_dividends(folder):
    #Creates a list of all files in folder
    filelist = os.listdir(folder)
    #Creates a empty list to append all dataframes
    dataframe_list = []
        #Iterates through all files in filelist
    for file in filelist:
        #Checks if file is a .csv
        if file.lower().endswith('.csv'):
            # Creates a relative path from working directory to the current element of filelist
            file_path = os.path.join('IbActivityNew', file)
            # reads file and creates a dataframe forcing 17 columns
            data = pd.read_csv(file_path, names=list('abcdefghijklmnopq'))
            # Dividend accruals happen more than once and cancel each other, so not very useful to get data, but fees
            # only appear there, and taxes aand dividends are on the same row, the dividend accrual line with the actual dividend 
            # payment will ocurr when a 'Dividens' row exists, so both dataframes are created, data cleaned and merged to get the
            # the correct dividend
            # # BUG ## Fees still not appear
            # Filters first column 'Dividends' values
            data_dividends = data[data.a == "Dividends"]
            # Filters first column 'Change in Dividend Accruals' values
            data_dividend_accruals = data[data.a == "Change in Dividend Accruals"]
            #Checks if any Dividends on file
            if len(data_dividends) > 0:
                # Removes extra rows
                data_dividends = data_dividends.dropna(subset=['e'])
                # Removes extra columns
                data_dividends = data_dividends.drop(columns=['a','b','g','h','i','j','k','l','m','n','o','p','q'])
                # Copies first rows values to columns names
                data_dividends.columns = data_dividends.iloc[0]
                # Remove first row
                data_dividends = data_dividends.drop(data_dividends.index[0])
                # Splits 'Description' column to get symbol
                data_dividends['Description'] = data_dividends['Description'].str.split('(', expand=True)
                data_dividends['Description'] = data_dividends['Description'].str.strip()
                # Same data cleaning as with 'Dividends' 
                data_dividend_accruals = data_dividend_accruals.dropna(subset=['h'])
                data_dividend_accruals = data_dividend_accruals.drop(columns=['a','b','c','f','g','i','l','o','p','q'])
                data_dividend_accruals.columns = data_dividend_accruals.iloc[0]
                data_dividend_accruals = data_dividend_accruals.drop(data_dividend_accruals.index[0])
                # Merge to get rows from Dividend Accruals matching 'Dividens' (Real payments)
                dividends = data_dividend_accruals.merge(data_dividends, left_on=['Currency','Symbol','Pay Date'],right_on=['Currency','Description','Date'], how='left')
                # Remove extra rows
                dividends = dividends.dropna(subset=['Amount'])
                # Remove extra columns
                dividends = dividends.drop(columns=['Date','Gross Amount','Description','Amount'])
                # Make Net Amount positive for credits and negative for debits
                dividends['Net Amount'] = dividends['Net Amount'].astype('float64') * -1
                # Append the dataframe to list of dataframes
                dataframe_list.append(dividends)
    # Concattenates all dataframes         
    dataframe = pd.concat(dataframe_list,sort=False ).reset_index(drop=True)
    dataframe['Tax'] = dataframe['Tax'].astype('float')
    dataframe['Fee'] = dataframe['Fee'].astype('float')
    dataframe['Net Amount'] = dataframe['Net Amount'].astype('float')
    dataframe['Pay Date'] = pd.to_datetime(dataframe['Pay Date'])
    
    return dataframe


In [26]:
dividends_df = get_all_dividends(folder)

In [27]:
dividends_df

,Currency,Symbol,Pay Date,Tax,Fee,Net Amount
0,USD,VZ,2017-05-01,-17.33,0.0,40.43
1,USD,SBUX,2017-05-26,-0.75,0.0,1.75
2,USD,IBKR,2017-06-14,-3.00,0.0,7.00
3,USD,VZ,2017-08-01,-17.33,0.0,40.43
4,USD,SBUX,2017-08-25,-0.75,0.0,1.75
...,...,...,...,...,...,...
108,USD,AAPL,2019-08-15,-3.70,0.0,8.62
109,USD,CEFL,2019-08-20,0.00,0.0,22.77
110,USD,MORL,2019-08-20,0.00,0.0,10.42
111,EUR,VEON,2019-08-23,-4.13,0.0,23.40


In [0]:
#Save .csv
#dividends_df.to_csv((os.path.join(tables,r'tables_dividends.csv')))

In [30]:
def get_all_fees(folder):
    #Creates a list of all files in folder
    filelist = os.listdir(folder)
    #Creates a empty list to append all dataframes
    dataframe_list = []
        #Iterates through all files in filelist
    for file in filelist:
        #Checks if file is a .csv
        if file.lower().endswith('.csv'):
            # Creates a relative path from working directory to the current element of filelist
            file_path = os.path.join('IbActivityNew', file)
            # reads file and creates a dataframe forcing 17 columns
            data = pd.read_csv(file_path, names=list('abcdefghijklmnopq'))
            # # BUG ## Fees still not appear
            # Filters first column 'Dividends' values
            data_fees = data[data.a == "Fees"]
            if len(data_fees) > 0:
                data_fees = data_fees.dropna(subset=['e'])
                data_fees = data_fees.drop(columns=['a','b','c','h','i','j','k','l','m','n','o','p','q'])
                data_fees.columns = data_fees.iloc[0]
                data_fees = data_fees.drop(data_fees.index[0])
                dataframe_list.append(data_fees)
    dataframe = pd.concat(dataframe_list,sort=False )
    return dataframe

In [31]:
fees_df = get_all_fees(folder)

In [32]:
fees_df

,Currency,Date,Description,Amount
114,USD,2017-04-04,AMEX Level I for Mar 2017,-1.5
115,USD,2017-04-04,OPRA (US Options Exchanges) Non-Professional L...,-1.5
129,USD,2017-05-03,AMEX Level I for Apr 2017,-1.5
130,USD,2017-05-03,BME (MEFF) Non-Professional Level I for Apr 2017,-2.45
131,USD,2017-05-03,NYSE Level I for Apr 2017,-1.5
...,...,...,...,...
572,USD,2019-08-05,J******17:HKFE L1 FOR AUG 2019,-3.19
573,USD,2019-08-05,j******17:US Securities Snapshot and Futures V...,10
574,USD,2019-08-05,j******17:US Securities Snapshot and Futures V...,-10
575,USD,2019-08-05,J******17:STUTTGART NP L1 FOR AUG 2019,-6.09


In [0]:
fees_df.to_csv((os.path.join(tables,r'tables_fees.csv')))

## Notes
- ADR fees on dividend accruals on file are also are on fee. Included here in tables_fees 


## Work in Progress
+ Make a dictionary of hash files 
- check if new files in dictionary and process 
- save to a file
- ADR fee https://ibkr.info/article/1185

### Relative paths with os.path.join

In [8]:
os.getcwd()

'd:\\Javier\\SynologyDrive\\Portfolio Python\\test'